# Method Chaining

Remember that _pandas_ snippet from way at the beginning of this tutorial? 

```
dataframe.groupby("some_field”).size().rename("new_name").reset_index()
```

That style of syntax is known as _method chaining_. 

Now that you're armed with the knowledge that _methods_ are basically functions that live in classes (and instances of those classes), you can begin to make sense of the phrase "method chaining": It's a technique that allows you to consecutively call methods, one after another, _without having to store and operate on the return value of each step in the chain._

There's one additional concept that's required to make sense of such code.

Similar to Python functions, methods can explicitly `return` a value.

Let's construct a new class to prove the point. Here we'll introduce a special method called [\_\_init\_\_](https://docs.python.org/3/tutorial/classes.html#class-objects) that you can use to add data attributes to an instance when you first create it. The syntax is a bit gnarly, but a simple demo should hopefully make its purpose clear.

In [ ]:
class Number:
    
    def __init__(self, number):
        # Store the number in "value" when you create the instance
        self.value = number
        
    def add(self, other_number):
        # Add our original number (stored in self.value) to some other_number
        # and return the solution
        return self.value + other_number

**Important**: Note that above, our `add` method plucks its value from...well...the `value` attribute, which is stored when we create the instance. The `add` method then adds its own stored value to `other_number` and returns the solution.

Ok, let's create an instance of `Number`. Note that because our special `__init__` method requires a number argument, we must pass this argument when we create the instance.

In [ ]:
num = Number(2) # The parens are reminiscent of a function call, right?
num.value

We can see that by passing `2` to the `Number` class when we create the instance, the number gets stored in the `value` attribute. Now let's do some addition.

In [ ]:
num.add(3) # Add 3 to our number

Ok, now that you have a sense of `self` -- pun intended -- and the fact that methods can return values, you're ready for one last concept that will help you grok the "method chaining" syntax.


Let's say that we wanted a number class that (1) was able to update itself **_and_** (2) allow you to perform multiple consecutive operations -- all without having to individually store and operate on the value in each step.

The first requirement is pretty straight-forward. We could update the `add` method to simply store the new solution in the `value` attribute:


```python
    def add(self, other_number):
        # Replace the original value with the newly calculated value
        self.value = self.value + other_number
        # Return the updated value
        return self.value
```

Let's create a new class using this approach.

In [ ]:
class FancyNumber:
    
    def __init__(self, number):
        self.value = number
        
    def add(self, other_number):
        # Replace the original value with the newly calculated value
        self.value = self.value + other_number
        # Return the updated value
        return self.value

In [ ]:
num = FancyNumber(1)
num.add(1)

Let's confirm the underlying value of our number has changed from `1` to `2`.

In [ ]:
num.value # should now be 2

We could continue updating the number by calling `num.add`:

In [ ]:
num.add(1)
num.add(1)
num.value # should now be 4

But Pythonistas and coders in general are allergic to keystrokes and visual clutter. Wouldn't it be nice if we could simply reference `num` a single time, and then just call `add` repeatedly? Let's try it:

In [ ]:
num = FancyNumber(0)
num.value

In [ ]:
num.add(1).add(1)

Ruh roh! Python got angry at us!

Read the error message carefully. It states that the `int` object has no attribute called `add`. 

Now look back at our `FancyNumber` class. Notice the `add` method is returning an integer (ie the sum of the original value and some other number)?

We already know how classes can serve as containers for related _attributes_ in the form of data and methods. Unfortunately for us, Python's built-in integer data type does not have a method called `add`. Don't believe us? Check out [the docs](https://docs.python.org/3/library/stdtypes.html#numeric-types-int-float-complex).

So how do we fix this situation? We clearly need to return something other than an integer in order to implement method chaining on our `FancyNumber` class.

Let's take stock of some key concepts:

1. Methods can return values
2. Python uses the `self` argument to reference specific instances of a class

So what if, instead of returning an integer, our `add` method simply returned itself (ie the instance of the class). Let's create one last version of the class and see if it works.



In [ ]:
# A number class that supports method chaining
class FanciestNumber:
    
    def __init__(self, number):
        self.value = number
        
    def add(self, other_number):
        # Replace the original value with the newly calculated value
        self.value = self.value + other_number
        # Return the instance of the class (NOT its current value)
        return self

In [ ]:
num = FanciestNumber(1)

In [ ]:
num.add(1).add(1)
num.value # This should be 3

That did the trick!! We now have a number that updates itself and allows us to repeatedly call the `add` method.

It's important to note that you can use method chaining with different types of objects. Stated differently, the methods you're chaining don't all have to live on the same class. For example, we could have done some basic method chaining using our very first implementation of `Number`:

In [ ]:
num = Number(1)
num.add(1).bit_count()

Above, the `add` method is of course on our original `Number` class. But the `bit_count` method is a (not so frequently used) method on integers. The important point is that **when you encounter (or use) method chaining, it's critical that you remain aware of the return value at every point in the chain.**

If you're ever in doubt, you can rewrite the code to use individual steps. In fact, it can be helpful to apply this approach when first writing the code. Once you're confident the code works as expected, you can _then_ rewrite it into a more compact form using method chaining.

In [ ]:
num = Number(2)
value = num.add(1)
value.bit_count()

# You can the rewrite the above as num.add(1).bit_count()

Folks who use Python libraries such as [pandas](https://pandas.pydata.org/docs/user_guide/index.html) -- e.g. its [DataFrame class](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) -- use method chaining extensively as a way to help reduce clutter. 

Remember that gnarly one-liner?

```
dataframe.groupby("some_field”).size().rename("new_name").reset_index()
```

You _could_ rewrite this snippet as a series of steps:

```
grouped = dataframe.groupby('some_field')
sized = grouped.size()
renamed = sized.rename("some_name")
df = renamed.reset_index()
df
```

But with method chaining, you can avoid having to create intermediate variables and just perform multiple operations on each version of the DataFrame instance (or whatever the return value is at a given point in the chain).